In [ ]:
# Cell 1: Project root setup (required for imports)

import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

In [ ]:
# Cell 2: Import project modules
# If running on Colab, make sure the src/ folder is available (upload or git clone)

from src.dataset import XRayDataLoaders
from src.model import DenseNet169Modified
from src.train import train_engine
from src.evaluate import evaluate_model
from src.utils import get_device, save_model, plot_training_curves

In [ ]:
# Cell 3: Configuration

DATA_DIR = "/tmp/XRAY"  # Path to dataset root directory
BATCH_SIZE = 32
DEVICE = get_device()

In [ ]:
# Cell 4: Prepare datasets and dataloaders
# Assumes the dataset has already been extracted to DATA_DIR

loader_manager = XRayDataLoaders(
    DATA_DIR,
    batch_size=BATCH_SIZE
)

train_dl, val_dl, test_dl = loader_manager.get_dataloaders()
class_names = train_dl.dataset.classes

In [ ]:
# Cell 5: Initialize model

model = DenseNet169Modified(num_classes=len(class_names))

In [ ]:
# Cell 6: Train model

history = train_engine(
    model,
    train_dl,
    val_dl,
    DEVICE,
    epochs=20,
    patience=5
)

In [ ]:
# Cell 7: Save trained model

save_model(model, "chest_xray_densenet169.pth")

In [ ]:
# Cell 8: Results and final evaluation

import matplotlib.pyplot as plt
import seaborn as sns

# 1) Training curves
fig = plot_training_curves(history)
fig.tight_layout()
plt.show()

# 2) Test set evaluation
results = evaluate_model(model, test_dl, DEVICE, class_names)

print(f"Test Accuracy: {results['accuracy']:.3f}")

# 3) Confusion matrix visualization
sns.heatmap(
    results["confusion_matrix"],
    annot=True,
    fmt="d",
    xticklabels=class_names,
    yticklabels=class_names
)

plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
